In [1]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
from torch.utils.data import (Dataset,DataLoader,) 
import pandas as pd
import os
from skimage import io

In [7]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=1):
        super(CNN, self).__init__()
        
        #First CNN
        self.conv1 = nn.Conv2d(
            in_channels = in_channels,
            out_channels= 6,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
    
        
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        #Second CNN
        self.conv2 = nn.Conv2d(
            in_channels= 6,
            out_channels=12,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        
        self.fc1 = nn.Linear(12 * 6 * 6, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        print("conv 1")
        print(x.shape)
        x = self.pool(x)
        print("pool 1")
        print(x.shape)
        x = F.relu(self.conv2(x))
        print("conv 2")
        print(x.shape)
        x = self.pool(x)
        print("pool 2")
        print(x.shape)
        # print("Shape 1")
        # print(x.shape)
        # print(x.shape[0])
        # print(x.shape[1])
        x = x.reshape(x.shape[0], -1)
        # print("Shape 2")
        # print(x.shape)
        # print(x.shape[0])
        # print(x.shape[1])
        x = self.fc1(x)
        return x

# model = CNN()
# x = torch.rand(64, 1, 28, 28)
# print(x)
# print(model(x).shape)
# # Set device
class Dataset_treatment(Dataset):
    def __init__(self,csv_file , root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 3
num_classes = 5
learning_rate = 0.001
batch_size = 32
num_epochs = 10


# Load Data
dataset = Dataset_treatment(    
    csv_file="animalsprueba.csv",
    root_dir="dataset",
    transform = transforms.ToTensor(),
)
print(len(dataset))

# Dataset is actually a lot larger ~25k images, just took out 10 pictures
# to upload to Github. It's enough to understand the structure and scale
# if you got more images.
# Load Data
train_set, test_set = torch.utils.data.random_split(dataset, [25,25])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Initialize network
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

50


  0%|          | 0/1 [00:00<?, ?it/s]

conv 1
torch.Size([25, 6, 300, 300])
pool 1
torch.Size([25, 6, 150, 150])
conv 2
torch.Size([25, 12, 150, 150])
pool 2
torch.Size([25, 12, 75, 75])


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [6]:
#Numero de OutputsFeatures Module
def OutputsFeatures(n_inputs , padding, kernel , stride):
    return ((n_inputs + 2 * padding - kernel)/stride)+1

OutputsFeatures(6,1,3,1)

6.0

In [1]:
#TRAINING (EPOCHS = 10)
#Accuracy on training set: 99.01
#Accuracy on test set: 98.68

#TRAINING 2 (EPOCHS = 5)
#Accuracy on training set: 99.01
#Accuracy on test set: 98.68

"""
Example of how to create custom dataset in Pytorch. In this case
we have images of cats and dogs in a separate folder and a csv
file containing the name to the jpg file as well as the target
label (0 for cat, 1 for dog).

Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
*    2020-04-03 Initial coding

"""

# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches


class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 5
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# Load Data
dataset = CatsAndDogsDataset(
    csv_file="animalsprueba.csv",
    root_dir="dataset",
    transform=transforms.ToTensor(),
)

# Dataset is actually a lot larger ~25k images, just took out 10 pictures
# to upload to Github. It's enough to understand the structure and scale
# if you got more images.
train_set, test_set = torch.utils.data.random_split(dataset, [25 , 25] )
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)



Cost at epoch 0 is 8.047879219055176
Cost at epoch 1 is 2.6225969791412354
Cost at epoch 2 is 0.4191370904445648
Cost at epoch 3 is 0.09029290080070496
Cost at epoch 4 is 0.01177942380309105
Cost at epoch 5 is 0.004528052639216185
Cost at epoch 6 is 0.002330646850168705
Cost at epoch 7 is 0.0010831287363544106
Cost at epoch 8 is 0.0007462475332431495
Cost at epoch 9 is 0.0008502152631990612
Checking accuracy on Training Set
Got 25 / 25 with accuracy 100.00
Checking accuracy on Test Set
Got 11 / 25 with accuracy 44.00
